# constants

In [12]:
FORMAT = 'svg'
RADIUS = 2000.
PATH = r'/Users/aleksandrvatkin/Desktop/work/ntk/Скважины-кусты_Спорышевское мр..xlsx'
BUSH_COST = 100 # mln rubles
BASE_DRILL_COST = 0
DRILL_METER_COST = 1
MAX_NUM_OF_WELLS_ON_BUSH = 12

# main code

In [13]:
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt


from tqdm import tqdm
from math import ceil
from functools import partial
from scipy.spatial import Delaunay
from collections import namedtuple, Counter
from itertools import product, combinations, chain
from mip import Model, BINARY, xsum, LinExpr, Var, minimize
from networkx.algorithms.community import louvain_communities as louvain

mpl.rcParams['savefig.format'] = FORMAT

class MyMinMaxScaler:
    __slots__ = ['min', 'max']

    def __init__(self):
        self.max = np.inf
        self.min = np.array([-np.inf])

    def fit(self, points):
        self.min = np.min(points, axis=0)
        self.max = max(np.max(points, axis=0) - self.min)

    def transform(self, points):
        return np.apply_along_axis(lambda point: (point - self.min) / self.max, 1, points)

    def fit_transform(self, points):
        self.fit(points)

        return self.transform(points)

    def reset_radius(self, radius):
        return radius / self.max


def euclidean_distance(first_point, second_point, power):
    return (np.sum(np.fabs(first_point - second_point)**power))**(1 / power)


def get_edge(first_id, first_point, second_id, second_point, radius, power):
    if euclidean_distance(first_point, second_point, power) <= radius:
        return first_id, second_id


def build_graph(ids, points, radius, power):
    graph = nx.Graph()
    graph.add_nodes_from(ids)
    edges_generator = filter(lambda x: x is not None,
                             (get_edge(*first_info, *second_info, radius, power)
                              for first_info, second_info in combinations(zip(ids, points), 2)))
    graph.add_edges_from(list(edges_generator))

    return graph


def get_cluster_ids(ids, communities):
    tmp_color_dict = {}

    for color_id, community in enumerate(communities):
        tmp_color_dict.update(dict(zip(community, [color_id] * len(community))))

    return [tmp_color_dict[_id] for _id in ids]


dist = partial(euclidean_distance, power=2)


def get_all_distances(wells):
    return [dist(first_point, second_point) for first_point, second_point in combinations(wells, 2)]


def get_grid_step(wells):
    distances = get_all_distances(wells)

    return np.median(distances) / 4


def get_bushes_coordinates(wells):
    min_x, min_y = np.min(wells, axis=0)
    max_x, max_y = np.max(wells, axis=0)
    grid_step = get_grid_step(wells)
    x_num_points = ceil((max_x - min_x) / grid_step)
    y_num_points = ceil((max_y - min_y) / grid_step)
    bush_coords = np.array(
        list(product(np.linspace(min_x, max_x, x_num_points), np.linspace(min_y, max_y, y_num_points))))
    convex_hull = Delaunay(wells)
    bush_coords = bush_coords[convex_hull.find_simplex(bush_coords) >= 0]

    return bush_coords


def get_nearest_points_ids(wells_df, bush_coords):
    nearest_points = []
    wells = wells_df.loc[:, ['normalized_x', 'normalized_y']].values

    for curr_bush in bush_coords:
        distances = np.array([dist(curr_bush, well) for well in wells])
        indices = np.argsort(distances)[:MAX_NUM_OF_WELLS_ON_BUSH]
        nearest_points.append(wells_df.iloc[indices].id.values)

    return nearest_points

In [14]:
Bush = namedtuple('Bush', ['name', 'coords', 'cost', 'mip_variable'])
Well = namedtuple('Well', ['name', 'coords', 'cost', 'mip_variable'])

In [15]:
def get_cost_of_well(distance):
    global BASE_DRILL_COST
    global DRILL_METER_COST
    
    return DRILL_METER_COST * distance + BASE_DRILL_COST


def wrapper():
    global MAX_NUM_OF_WELLS_ON_BUSH
    A = [(*x, -1 if sum(x) == MAX_NUM_OF_WELLS_ON_BUSH else 1) 
         for x in product((1, -1), repeat=MAX_NUM_OF_WELLS_ON_BUSH)]
    b = [-1 * Counter(x)[-1] + 0.1 for x in A]
    
    def _generate_wells_to_bush_constraint(bushes_wells, bush):
        variables = chain(map(lambda x: x.mip_variable, bushes_wells), [bush.mip_variable])
        constraints = [xsum((c * variable for c, variable in zip(c, variables))) >= right for c, right in zip(A, b)]
        
        return constraints
    
    return _generate_wells_to_bush_constraint


def get_all_wells_used_constraint(all_variables, number_of_wells):
    filtered_variables = filter(lambda x: isinstance(x, Well), all_variables)
    lin_expr = (xsum(map(lambda x: x.mip_variable, filtered_variables)) == number_of_wells)

    return lin_expr

                    
generate_wells_to_bush_constraint = wrapper()


def get_objective(all_variables):
    coefficients = map(lambda x: x.cost, all_variables)
    mip_variables = map(lambda x: x.mip_variable, all_variables)
    
    return minimize(xsum(coefficient * variable for coefficient, variable in zip(coefficients, variables)))


def add_constraints(model, constraints):
    for constraint in constraints:
        model += constraint
        
        
def add_objective(model, objective):
    model.objective = objective

In [16]:
def base_worker(data):
    wells = data.loc[:, ['normalized_x', 'normalized_y']].values
    bush_coords = get_bushes_coordinates(wells)
    all_nearest_points_ids = get_nearest_points_ids(data, bush_coords)
    model = Model()
    all_variables = []
    all_constraints = []
    
    for bush_id, bush_x_y, nearest_points_ids in tqdm(zip(range(len(bush_coords)), bush_coords, all_nearest_points_ids), 
                                                      total=len(bush_coords)):
        curr_bushes_wells = [None] * MAX_NUM_OF_WELLS_ON_BUSH
        bush_name = f'bush_{bush_id}'
        bush_variable = model.add_var(name=bush_name, var_type=BINARY)
        bush = Bush(name=bush_name, 
                    coords=bush_x_y, 
                    cost=BUSH_COST, 
                    mip_variable=bush_variable)
        all_variables.append(bush)
        
        nearest_points_coords = data[data.id.isin(nearest_points_ids)].loc[:, ['normalized_x', 'normalized_y']]
        
        for id_in_storage, point_id, point in zip(range(MAX_NUM_OF_WELLS_ON_BUSH), 
                                                  nearest_points_ids, nearest_points_coords):
            distance = dist(bush_x_y, point)
            well_cost = get_cost_of_well(distance)
            well_name = bush_name + '_' + str(point_id)
            well_variable = model.add_var(name=well_name, var_type=BINARY)
            well = Well(name=well_name,
                       coords=point,
                       cost=well_cost,
                       mip_variable=well_variable)
            all_variables.append(well)
            curr_bushes_wells[id_in_storage] = well
        
        all_constraints.extend(generate_wells_to_bush_constraint(curr_bushes_wells, bush))
        
    all_constraints.append(get_all_wells_used_constraint(all_variables, len(data)))
    
    add_constraints(model, all_constraints)
    add_objective(model, get_objective(all_variables))
    model.clique_merge()
    model.optimize()
    
    return all_variables

In [17]:
def read_data():
    data = pd.read_excel(PATH, engine='openpyxl')
    data.columns = ['id', 'bush', 'x', 'y', 'deposit']

    return data


def get_louvain_clusters(ids, points):
    graph = build_graph(ids, points, RADIUS, 1)
    communities = louvain(graph)

    return get_cluster_ids(ids, communities)


def get_default_clusters(ids, points):
    return [0] * len(ids)


def clusterization_worker(get_clusters):
    data = read_data()

    scaler = MyMinMaxScaler()
    points = scaler.fit_transform(data.loc[:, ['x', 'y']].values)
    global RADIUS
    RADIUS = scaler.reset_radius(RADIUS)

    data['cluster_id'] = get_clusters(data.id.values, points)
    data['normalized_x'] = points[:, 0]
    data['normalized_y'] = points[:, 1]

    clusters = set(data.cluster_id.values)

    for cluster_id in clusters:
        wells_df = data[data.cluster_id == cluster_id]
        all_variables = base_worker(wells_df)


clusterization_worker(get_louvain_clusters)

  0%|          | 0/55 [00:00<?, ?it/s]


UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('float64'), dtype('<U12')) -> None